In [1]:
import os
import glob
import geopandas as gpd
import numpy as np
from shapely.geometry import Point

In [2]:
# Prepare input data files
input_data = glob.glob('../data/raw/*.geojson')
data_type = ['bus', 'immo', 'commerces', 'ecoles', 'tram', 'velo']
dict_input_data = {i:j for i,j in zip(data_type, input_data)}

In [3]:
df_immo = gpd.read_file(dict_input_data['immo'], crs={"init":"EPSG:4326"})
df_immo[['x', 'y']] = df_immo.get_coordinates(ignore_index=True)

In [4]:
df_bus = gpd.read_file(dict_input_data['bus'], crs={"init":"EPSG:4326"})
df_bus[['x', 'y']] = df_bus.get_coordinates(ignore_index=True)

In [5]:
df_tram = gpd.read_file(dict_input_data['tram'], crs={"init":"EPSG:4326"})
df_tram[['x', 'y']] = df_tram.get_coordinates(ignore_index=True)

In [6]:
df_ecoles = gpd.read_file(dict_input_data['ecoles'], crs={"init":"EPSG:4326"})
df_ecoles[['x', 'y']] = df_ecoles.get_coordinates(ignore_index=True)

In [7]:
df_commerces = gpd.read_file(dict_input_data['commerces'], crs={"init":"EPSG:4326"})
df_commerces[['x', 'y']] = df_commerces.get_coordinates(ignore_index=True)

In [8]:
houses = df_immo.to_crs(epsg=4326)
shops = df_commerces.to_crs(epsg=4326)

In [18]:
# Create a 100m buffer around each house
houses_buffer = houses.buffer(500/111120)

# Convert the buffered object back to a GeoDataFrame
houses_buffer_gdf = gpd.GeoDataFrame(geometry=gpd.GeoSeries(houses_buffer), crs=houses.crs)

# Perform a spatial join to find shops within 100m of houses
shops_near_houses = gpd.sjoin(shops, 
                              houses_buffer_gdf, 
                              how="inner", 
                              predicate="intersects")

/tmp/ipykernel_28249/2713299841.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  houses_buffer = houses.buffer(500/111120)


In [19]:
shops_near_houses = shops_near_houses.reset_index()

In [22]:
shops_near_houses.groupby('index_right')['nom_enseigne'].count()

index_right
2          5
3         39
4         79
5        157
7        689
        ... 
39681     21
39683    191
39684     93
39685    383
39686    111
Name: nom_enseigne, Length: 29840, dtype: int64

In [8]:
df_immo['min_bus_id'] = df_immo.geometry.apply(lambda g: df_bus.distance(g).idxmin())

/tmp/ipykernel_15209/3693189071.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df_immo['min_bus_id'] = df_immo.geometry.apply(lambda g: df_bus.distance(g).idxmin())


In [9]:
df_immo['min_tram_id'] = df_immo.geometry.apply(lambda g: df_tram.distance(g).idxmin())

/tmp/ipykernel_15209/2405465780.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df_immo['min_tram_id'] = df_immo.geometry.apply(lambda g: df_tram.distance(g).idxmin())


In [10]:
df_immo['min_ecole_id'] = df_immo.geometry.apply(lambda g: df_ecoles.distance(g).idxmin())

/tmp/ipykernel_15209/3192144990.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df_immo['min_ecole_id'] = df_immo.geometry.apply(lambda g: df_ecoles.distance(g).idxmin())


In [14]:
df_immo.head().T

,0,1,2,3,4
libnatmut,Vente,Vente,Vente,Vente,Vente
type_bien,Maison,Maison,Maison,Appartement,Appartement
typologie,4 pièces,5 pièces ou plus,4 pièces,T3,T4
valeurfonc,237500.0,440000.0,192500.0,38000.0,107500.0
prix_m2,1532.26,2716.05,2750.0,593.75,1733.87
commune,Quetigny,Dijon,Dijon,Dijon,Dijon
code_insee,21515,21231,21231,21231,21231
quartier,None,Fontaine d'Ouche - Faubourg Raines - Larrey - ...,Bourroches - Port du Canal - Valendons - Monta...,Fontaine d'Ouche - Faubourg Raines - Larrey - ...,Maladiere - Drapeau - Clemenceau
code_quartier,None,8,7,8,3
nom_iris,Atrias-Vieux Village-Grand Chaignet,La Montagne,Les Valendons,Tire-Pesseau,Clemenceau


In [50]:
radius_meters = 500

nb_commerces_100m = gpd.sjoin_nearest(df_immo, 
                                            df_commerces[['geometry','nom_enseigne']], 
                                            #distance_col="distance", 
                                      how='left',
                                            max_distance=radius_meters)
#nb_commerces_100m=nb_commerces_100m.groupby('index_right')['nom_enseigne'].count()

/home/gael/miniconda3/envs/pandas/lib/python3.10/site-packages/geopandas/array.py:365: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


In [51]:
nb_commerces_100m.sort_index().head().T

,0,1,2,3,4
libnatmut,Vente,Vente,Vente,Vente,Vente
type_bien,Maison,Maison,Maison,Appartement,Appartement
typologie,4 pièces,5 pièces ou plus,4 pièces,T3,T4
valeurfonc,237500.0,440000.0,192500.0,38000.0,107500.0
prix_m2,1532.26,2716.05,2750.0,593.75,1733.87
commune,Quetigny,Dijon,Dijon,Dijon,Dijon
code_insee,21515,21231,21231,21231,21231
quartier,None,Fontaine d'Ouche - Faubourg Raines - Larrey - ...,Bourroches - Port du Canal - Valendons - Monta...,Fontaine d'Ouche - Faubourg Raines - Larrey - ...,Maladiere - Drapeau - Clemenceau
code_quartier,None,8,7,8,3
nom_iris,Atrias-Vieux Village-Grand Chaignet,La Montagne,Les Valendons,Tire-Pesseau,Clemenceau


In [44]:
df_commerces

,nom_enseigne,disponibilte,dashboard_vacance,num_adresse,adresse,nom_quartier,surface,rubrique,secteur_activite,independant_enseigne,date_maj,geo_point_2d,geometry,x,y
0,Le Dressing Infidèle,Vendu,En activité,38,RUE DE JOUVENCE,Montchapet,NaN,Habillement et chaussures,Dépôt vente,Independant,2022-02-10 14:46:06+00:00,None,POINT (5.03855 47.33152),5.038553,47.331520
1,Atelier Lionel verrier,Vendu,En activité,12,RUE JEAN RENOIR,Centre-Ville,NaN,"Culture, loisirs, sports",Galerie d'art,Independant,2018-05-02 15:22:01+00:00,None,POINT (5.03436 47.31440),5.034362,47.314398
2,Boulangerie Oudot,Vendu,En activité,43,AVENUE RAYMOND POINCARE,Grésilles,139.0,Alimentation,Boulangerie,Independant,2016-07-20 02:00:00+00:00,None,POINT (5.06587 47.32911),5.065871,47.329109
3,YE SUSHI,Vendu,En activité,47,AVENUE RAYMOND POINCARE,Grésilles,141.0,CHR,Restaurant,Independant,2022-06-22 02:00:00+00:00,None,POINT (5.06611 47.32911),5.066111,47.329111
4,Casa Del Burger,Vendu,En activité,19,RUE ANGELIQUE DUCOUDRAY,Université,NaN,CHR,Restauration rapide,Independant,2019-12-12 00:00:00+00:00,None,POINT (5.07111 47.32361),5.071107,47.323614
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2897,La belle étoffe,Vendu,En activité,5,RUE DE MAYENCE,Grésilles,474.0,Autres commerces de détail,Mercerie,Independant,2022-11-02 00:09:21+00:00,None,POINT (5.06210 47.34567),5.039796,47.321458
2898,Izac,Vendu,En activité,13,RUE DU BOURG,Centre-Ville,150.0,Habillement et chaussures,Mode homme,Enseigne,2023-03-25 00:00:00+00:00,None,POINT (5.03980 47.32146),5.061491,47.317363
2899,Crédit Mutuel Mirande,Vendu,En activité,65,RUE DE MIRANDE,Université,NaN,Services,Banque,Enseigne,2019-03-27 00:00:00+00:00,None,POINT (5.06149 47.31736),5.023090,47.307179
2900,Salon MORENA,Vendu,En activité,51,RUE DE CHENOVE,Bourroches - Port du Canal - Valendons - Monta...,NaN,Bien-être et beauté,Coiffeur,Independant,2019-10-10 00:00:00+00:00,None,POINT (5.02309 47.30718),5.043951,47.323237


In [23]:
radius_meters = 250

nb_commerces_250m = gpd.sjoin_nearest(df_commerces[['geometry','nom_enseigne']], 
                                            df_immo, 
                                            distance_col="distance", 
                                            max_distance=radius_meters)
nb_commerces_250m=nb_commerces_250m.groupby('index_right')['nom_enseigne'].count()

/home/gael/miniconda3/envs/pandas/lib/python3.10/site-packages/geopandas/array.py:365: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


In [27]:
radius_meters = 500

nb_commerces_500m = gpd.sjoin_nearest(df_commerces[['geometry','nom_enseigne']], 
                                            df_immo, 
                                            distance_col="distance", 
                                            max_distance=radius_meters)
#nb_commerces_500m=nb_commerces_500m.groupby('index_right')['nom_enseigne'].count()

/home/gael/miniconda3/envs/pandas/lib/python3.10/site-packages/geopandas/array.py:365: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


In [30]:
nb_commerces_500m.sort_index().iloc[3]

geometry         POINT (5.034362182937988 47.31439765965091)
nom_enseigne                          Atelier Lionel verrier
index_right                                            30069
libnatmut                                              Vente
type_bien                                        Appartement
typologie                                                 T1
valeurfonc                                            1700.0
prix_m2                                                 68.0
commune                                                Dijon
code_insee                                             21231
quartier                                        Centre-Ville
code_quartier                                              6
nom_iris                                       Petit Cîteaux
code_iris                                          212310401
datemut                                           2015-10-12
anneemut                                                2015
moismut                 

In [ ]:
https://towardsdatascience.com/optimising-the-location-of-food-delivery-couriers-using-geopandas-and-openstreetmap-e771433e5e9f

In [8]:
def find_nearby_shops(houses_gdf, shops_gdf, radius=100):
    # Create a copy of the shops dataframe
    nearby_shops = shops_gdf.copy()
    
    # Create a buffer around each house
    houses_gdf['geometry'] = houses_gdf.geometry.buffer(radius)
    
    # Iterate over each house
    for index, house in houses_gdf.iterrows():
        # Find shops that intersect with the buffer of the house
        nearby_shops['nearby'] = nearby_shops.intersects(house['geometry'])
        
        # Select nearby shops
        nearby_shops_subset = nearby_shops[nearby_shops['nearby']]
        
        
        # You can perform any other operation here, e.g., store the nearby shops in a list or a new dataframe
        
        # Drop the temporary 'nearby' column
        nearby_shops.drop(columns=['nearby'], inplace=True)
    
    # Reset the index of the nearby shops dataframe
    nearby_shops.reset_index(drop=True, inplace=True)
    
    return nearby_shops

In [33]:
df_immo['buffer'] = df_immo.geometry.buffer(0.001)

/tmp/ipykernel_22491/2552670408.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df_immo['buffer'] = df_immo.geometry.buffer(0.001)


In [39]:
df_immo.intersects(df_commerces)

/tmp/ipykernel_22491/2984173577.py:1: UserWarning: The indices of the two GeoSeries are different.
  df_immo.intersects(df_commerces)


0         True
1         True
2         True
3         True
4         True
         ...  
39682    False
39683    False
39684    False
39685    False
39686    False
Length: 39687, dtype: bool

In [9]:
nearby_shops = find_nearby_shops(df_immo, df_commerces)

/tmp/ipykernel_22491/1816758813.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  houses_gdf['geometry'] = houses_gdf.geometry.buffer(radius)


In [10]:
nearby_shops

,nom_enseigne,disponibilte,dashboard_vacance,num_adresse,adresse,nom_quartier,surface,rubrique,secteur_activite,independant_enseigne,date_maj,geo_point_2d,geometry,x,y
0,Le Dressing Infidèle,Vendu,En activité,38,RUE DE JOUVENCE,Montchapet,NaN,Habillement et chaussures,Dépôt vente,Independant,2022-02-10 14:46:06+00:00,None,POINT (5.03855 47.33152),5.038553,47.331520
1,Atelier Lionel verrier,Vendu,En activité,12,RUE JEAN RENOIR,Centre-Ville,NaN,"Culture, loisirs, sports",Galerie d'art,Independant,2018-05-02 15:22:01+00:00,None,POINT (5.03436 47.31440),5.034362,47.314398
2,Boulangerie Oudot,Vendu,En activité,43,AVENUE RAYMOND POINCARE,Grésilles,139.0,Alimentation,Boulangerie,Independant,2016-07-20 02:00:00+00:00,None,POINT (5.06587 47.32911),5.065871,47.329109
3,YE SUSHI,Vendu,En activité,47,AVENUE RAYMOND POINCARE,Grésilles,141.0,CHR,Restaurant,Independant,2022-06-22 02:00:00+00:00,None,POINT (5.06611 47.32911),5.066111,47.329111
4,Casa Del Burger,Vendu,En activité,19,RUE ANGELIQUE DUCOUDRAY,Université,NaN,CHR,Restauration rapide,Independant,2019-12-12 00:00:00+00:00,None,POINT (5.07111 47.32361),5.071107,47.323614
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2897,La belle étoffe,Vendu,En activité,5,RUE DE MAYENCE,Grésilles,474.0,Autres commerces de détail,Mercerie,Independant,2022-11-02 00:09:21+00:00,None,POINT (5.06210 47.34567),5.039796,47.321458
2898,Izac,Vendu,En activité,13,RUE DU BOURG,Centre-Ville,150.0,Habillement et chaussures,Mode homme,Enseigne,2023-03-25 00:00:00+00:00,None,POINT (5.03980 47.32146),5.061491,47.317363
2899,Crédit Mutuel Mirande,Vendu,En activité,65,RUE DE MIRANDE,Université,NaN,Services,Banque,Enseigne,2019-03-27 00:00:00+00:00,None,POINT (5.06149 47.31736),5.023090,47.307179
2900,Salon MORENA,Vendu,En activité,51,RUE DE CHENOVE,Bourroches - Port du Canal - Valendons - Monta...,NaN,Bien-être et beauté,Coiffeur,Independant,2019-10-10 00:00:00+00:00,None,POINT (5.02309 47.30718),5.043951,47.323237
